In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 
import datetime
import calendar
import statsmodels.api as sm

In [56]:
def clean_data(df):
    df.Date = pd.to_datetime(df.Date)

    df['day_of_week'] = df['Date'].dt.weekday_name

    df = df.drop(['High','Low','Volume'],axis=1)

    df['Consecutive trading day'] = df['day_of_week'].shift(1) + ' to ' + df['day_of_week']

    df['Overnight return'] = (df['Open'] - df['Adj Close'].shift(1)/df['Adj Close'].shift(1))*100
    
    df['Return'] = ((df['Close'] - df['Close'].shift(1))/df['Close'].shift(1))*100
    
    df = df.dropna()

    return df

In [46]:
def do_OLS(df):
    df = df.join(pd.get_dummies(df['day_of_week']))
    df['Difference in return'] = df['Return'] - df['Return'].shift(1) # include the lag value
    df = df.dropna()
    days = df[['Monday','Tuesday','Wednesday','Thursday','Friday']]
    days = sm.add_constant(days)

    OLS = sm.OLS(df['Difference in return'],days)
    
    result = OLS.fit()
    
    return result.params

In [59]:
# DIDN'T DO YET - Day of the week effect in Overnight Return 
def do_OLS_overnight(df):
    df = df.join(pd.get_dummies(df['Consecutive trading day']))
    df['Difference in overnight return'] = df['Overnight return'] - df['Overnight return'].shift(1)
    df = df.dropna()
    days = df[['Monday to Tuesday','Tuesday to Wednesday','Wednesday to Thursday','Thursday to Friday','Friday to Monday']]
    days = sm.add_constant(days)

    OLS = sm.OLS(df['Difference in overnight return'],days)
    
    result = OLS.fit()
    
    return result.params

## Day of the week effect in Return (difference in closing price) with Ordinary Least Square
- **1997 - 2007: GSPC, VIX**
- **2007 - end of 2008: GSPC, VIX**
- **2009 - 2017: GSPC, VIX, VXX **


### 1997 - 2007: 

In [57]:
GSPC_P1 = pd.read_csv('/Users/Lee/Desktop/research-1/^GSPC_1997-2007.csv')
GSPC_P1 = clean_data(GSPC_P1)
GSPC_P1_summary = do_OLS(GSPC_P1)

In [58]:
VIX_P1 = pd.read_csv('/Users/Lee/Desktop/research-1/^VIX_1997-2007.csv')
VIX_P1 = clean_data(VIX_P1)
VIX_P1_summary = do_OLS(VIX_P1)

In [59]:
d = {'GSPC':GSPC_P1_summary, 'VIX':VIX_P1_summary}
before_crisis = pd.DataFrame(d).transpose()

In [60]:
before_crisis

,const,Monday,Tuesday,Wednesday,Thursday,Friday
GSPC,-0.000310,0.030662,0.008044,-0.015268,-0.012074,-0.011674
VIX,0.048151,3.398603,-2.240443,-0.364558,0.489330,-1.234782


### 2007 - end of 2008

In [61]:
GSPC_P2 = pd.read_csv('/Users/Lee/Desktop/research-1/^GSPC_2007-2008.csv')
GSPC_P2 = clean_data(GSPC_P2)
GSPC_P2_summary = do_OLS(GSPC_P2)

In [62]:
VIX_P2 = pd.read_csv('/Users/Lee/Desktop/research-1/^VIX_2007-2008.csv')
VIX_P2 = clean_data(VIX_P2)
VIX_P2_summary = do_OLS(VIX_P2)

In [63]:
d_2 = {'GSPC':GSPC_P2_summary,'VIX':VIX_P2_summary}
during_crisis = pd.DataFrame(d_2).transpose()

In [64]:
during_crisis

,const,Monday,Tuesday,Wednesday,Thursday,Friday
GSPC,0.000975,-0.238782,0.418405,-0.416849,0.073553,0.164649
VIX,0.012624,1.248902,-2.106905,0.808401,0.391051,-0.328825


### 2009 - 2017 

In [40]:
GSPC_P3 = pd.read_csv('/Users/Lee/Desktop/research-1/^GSPC_2009-now.csv')
GSPC_P3 = clean_data(GSPC_P3)
GSPC_P3_summary = do_OLS(GSPC_P3)

In [41]:
VIX_P3 = pd.read_csv('/Users/Lee/Desktop/research-1/^VIX_2009-now.csv')
VIX_P3 = clean_data(VIX_P3)
VIX_P3_summary = do_OLS(VIX_P3)

In [42]:
VXX = pd.read_csv('/Users/Lee/Desktop/research-1/VXX_2009-now.csv')
VXX = clean_data(VXX)
VXX_summary = do_OLS(VXX)

In [43]:
d_3 = {'GSPC':GSPC_P3_summary, 'VIX':VIX_P3_summary, 'VXX':VXX_summary}
after_crisis = pd.DataFrame(d_3).transpose()

In [44]:
after_crisis

,const,Monday,Tuesday,Wednesday,Thursday,Friday
GSPC,-0.000048,0.006413,0.054503,0.004029,-0.013493,-0.051499
VIX,0.029061,1.932122,-1.094742,-0.170557,0.130796,-0.768558
VXX,-0.007254,-0.485737,0.170922,0.177322,0.029475,0.100763


### Summary:

We test the presence of day of the week effect in stock market return during the January 1997 and November 2017 periods by partition the periods into three periods because of the financial crisis in 2008: from January 1997 to January 2007, from January 2007 to the end of 2008, from 2009 to November 2017.


We use the S&P 500 Stock Index, S&P 500 Volatility Index (VIX) for all three periods, and iPath S&P 500 VIX Short-Term Futures Index (VXX) for the period since 2009. 


By employing the OLS model, we find for S&P Stock Index, before 2007, Monday has the highest return, Wednesday has the lowest return, from 2007 to 2008, Tuesday has the highest return, Wednesday has the lowest return, after 2008, Tuesday has the highest return, Friday has the lowest return. For VIX, Monday has the highest return and Tuesday has the lowest return regardless of the periods. For VXX, Wednesday has the highest return and Monday has the lowest return.